<a href="https://colab.research.google.com/github/ignaciakramm/analisis_linguistico_tesis-/blob/main/Proryecto_de_tesis_M%C3%A9tricas_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar librería de diversidad léxica
!pip install lexical-diversity

# Descargar el modelo grande de español (necesario para la coherencia/vectores)
!python -m spacy download es_core_news_lg
import spacy
import pandas as pd
import numpy as np
from lexical_diversity import lex_div as ld
import os
# Cargar modelo de español grande (necesario para vectores/word2vec)
nlp = spacy.load("es_core_news_lg")

# --- 1. CARGAR EL MODELO ---
print("Cargando modelo... (si falla aquí, recuerda reiniciar el entorno de ejecución)")
try:
    nlp = spacy.load("es_core_news_lg")
except OSError:
    print("ERROR: No se encuentra el modelo. Asegúrate de haber ejecutado la celda de instalación y REINICIADO el entorno.")
    raise




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 936.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Cargando modelo... (si falla aquí, recuerda reiniciar el entorno de ejecución)


In [4]:
def calcular_metricas(texto_paciente, id_participante, grupo, duracion_minutos=1):
    # NOTA: Agregué 'duracion_minutos' por defecto en 1 si no tienes el dato real

    doc = nlp(texto_paciente)

    # 1. Palabras y Tasa
    num_palabras = len(doc)
    tasa_produccion = num_palabras / duracion_minutos # Palabras por minuto

    # 2. Oraciones y MLU (Longitud media de enunciados)
    num_oraciones = len(list(doc.sents))
    promedio_long_oracion = num_palabras / num_oraciones if num_oraciones > 0 else 0

    # 3. Diversidad (MTLD)
    tokens_texto = [token.text for token in doc if not token.is_punct]
    try:
        mtld_val = ld.mtld(tokens_texto)
    except:
        mtld_val = 0

    # 4. Complejidad Sintáctica (Densidad Verbal)
    num_verbos = len([token for token in doc if token.pos_ == "VERB"])
    densidad_verbal = num_verbos / num_palabras if num_palabras > 0 else 0

    # 5. Cohesión Textual (Conectores) ---
    # Usamos las etiquetas POS de Spacy: CCONJ (Coordinantes) y SCONJ (Subordinantes)
    # Ejemplos: y, o, pero, ni, que, si, cuando, como, porque...
    num_conectores = len([token for token in doc if token.pos_ in ["CCONJ", "SCONJ"]])
    indice_cohesion = num_conectores / num_palabras if num_palabras > 0 else 0

    # 6. Marcadores Discursivos (Muletillas/Relleno) ---
    # Definimos una lista manual de marcadores típicos en español
    lista_marcadores = ["eh", "em", "este", "digamos", "o sea", "bueno", "mira", "sabes"]

    # Contamos cuántas veces aparecen
    texto_lower = texto_paciente.lower()
    num_marcadores = sum(texto_lower.count(m) for m in lista_marcadores)
    indice_marcadores = num_marcadores / num_palabras if num_palabras > 0 else 0

    # 7. Coherencia Semántica (Vectores)
    oraciones = list(doc.sents)
    similitudes = []
    for i in range(len(oraciones) - 1):
        if oraciones[i].vector_norm and oraciones[i+1].vector_norm:
            sim = oraciones[i].similarity(oraciones[i+1])
            similitudes.append(sim)
    coherencia = np.mean(similitudes) if len(similitudes) > 0 else 0

    return {
        "ID": id_participante,
        "Grupo": grupo,
        "Total_Palabras": num_palabras,
        "Tasa_Produccion_PPM": round(tasa_produccion, 2),
        "MLU_Long_Oracion": round(promedio_long_oracion, 2),
        "Diversidad_MTLD": round(mtld_val, 2),
        "Densidad_Verbal": round(densidad_verbal, 2),
        "Indice_Cohesion_Conectores": round(indice_cohesion, 3),
        "Indice_Marcadores": round(indice_marcadores, 3),
        "Coherencia_Semantica": round(coherencia, 4)
    }

In [2]:
# 1. Define la ruta a tu archivo en Drive
# NOTA: Cambia 'CarpetaTesis' y 'entrevista_1.txt' por tus nombres reales
ruta_archivo_drive = "/content/drive/MyDrive/corpus_tesis/entrevista_prueba.txt"

# 2. Verificamos que la ruta sea correcta antes de intentar leer
import os

if os.path.exists(ruta_archivo_drive):
    print(f"Archivo encontrado: {ruta_archivo_drive}")

    # 3. Leemos el contenido
    with open(ruta_archivo_drive, "r", encoding="utf-8") as f:
        contenido = f.read()

    # 4. Llamamos a la función (que ya definiste arriba)
    # Puedes cambiar el ID y Grupo manualmente o extraerlos del nombre del archivo
    resultados = calcular_metricas(contenido, id_participante="Sujeto_Drive", grupo="EMAR")

    # 5. Mostrar resultados
    import pandas as pd
    from IPython.display import display
    df = pd.DataFrame([resultados])
    display(df)

else:
    print(f"ERROR: No encuentro el archivo en: {ruta_archivo_drive}")
    print("Consejo: Ve a la carpeta de la izquierda, busca tu archivo, haz clic derecho y elige 'Copiar ruta' (Copy path).")

✅ Archivo encontrado: /content/drive/MyDrive/corpus_tesis/entrevista_prueba.txt


,ID,Grupo,Total_Palabras,Oraciones,Long_Media_Oracion,Diversidad_MTLD,Densidad_Verbal,Coherencia_Semantica
0,Sujeto_Drive,EMAR,3445,39,88.33,45.12,0.12,0.8263
